# Irisa Limited Access Test

This notebook tests the limited access for user 'irisa' who can only access ID and Amount columns.

In [ ]:
# Configuration for Irisa limited access
CATALOG_URL = "http://lakekeeper:8181/catalog"
KEYCLOAK_TOKEN_ENDPOINT = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"
WAREHOUSE = "irisa-ot"

# Irisa user credentials
CLIENT_ID = "irisa-client"
CLIENT_SECRET = "IrisaSecretKey123456789"
USERNAME = "irisa"
PASSWORD = "password123"  # This should match the password in Keycloak

In [ ]:
import requests
import json

# Get access token for Irisa user
def get_irisa_token():
    token_url = f"{KEYCLOAK_TOKEN_ENDPOINT}"
    
    # First get token using client credentials
    client_data = {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    }
    
    response = requests.post(token_url, data=client_data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print(f"Error getting client token: {response.text}")
        return None

token = get_irisa_token()
print(f"✅ Token obtained: {token[:50]}..." if token else "❌ Failed to get token")

In [ ]:
# Test 1: Try to access only ID and Amount columns (should work)
print("🔍 Test 1: Accessing ID and Amount columns (should work)")

headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

# Query with only allowed columns
query_data = {
    "sql": "SELECT ID, Amount FROM irisa.fake_seclink LIMIT 5"
}

response = requests.post(f"{CATALOG_URL}/query", headers=headers, json=query_data)
print(f"Status: {response.status_code}")
if response.status_code == 200:
    print("✅ Success! Irisa can access ID and Amount columns")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"❌ Failed: {response.text}")

In [ ]:
# Test 2: Try to access Source column (should be denied)
print("\n🔍 Test 2: Accessing Source column (should be denied)")

query_data = {
    "sql": "SELECT ID, Amount, Source FROM irisa.fake_seclink LIMIT 5"
}

response = requests.post(f"{CATALOG_URL}/query", headers=headers, json=query_data)
print(f"Status: {response.status_code}")
if response.status_code == 403:
    print("✅ Success! Access to Source column is properly denied")
    print(f"Error: {response.text}")
else:
    print(f"❌ Unexpected result: {response.text}")

In [ ]:
# Test 3: Try to access all columns (should be denied)
print("\n🔍 Test 3: Accessing all columns (should be denied)")

query_data = {
    "sql": "SELECT * FROM irisa.fake_seclink LIMIT 5"
}

response = requests.post(f"{CATALOG_URL}/query", headers=headers, json=query_data)
print(f"Status: {response.status_code}")
if response.status_code == 403:
    print("✅ Success! Access to all columns is properly denied")
    print(f"Error: {response.text}")
else:
    print(f"❌ Unexpected result: {response.text}")

In [ ]:
# Test 4: Compare with full access user (Trino service account)
print("\n🔍 Test 4: Comparing with full access user")

# Get token for Trino service account
trino_token_data = {
    'grant_type': 'client_credentials',
    'client_id': 'trino',
    'client_secret': 'AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ'
}

trino_response = requests.post(KEYCLOAK_TOKEN_ENDPOINT, data=trino_token_data)
if trino_response.status_code == 200:
    trino_token = trino_response.json()['access_token']
    
    trino_headers = {
        'Authorization': f'Bearer {trino_token}',
        'Content-Type': 'application/json'
    }
    
    # Trino should be able to access all columns
    query_data = {
        "sql": "SELECT * FROM irisa.fake_seclink LIMIT 5"
    }
    
    response = requests.post(f"{CATALOG_URL}/query", headers=trino_headers, json=query_data)
    print(f"Trino access status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Trino service account has full access as expected")
    else:
        print(f"❌ Trino access failed: {response.text}")
else:
    print(f"❌ Failed to get Trino token: {trino_response.text}")

## Summary

This test demonstrates:
1. ✅ Irisa user can access only ID and Amount columns
2. ✅ Access to other columns (Source, Destination, etc.) is properly denied
3. ✅ Full access users (Trino, StarRocks, DuckDB) maintain their privileges
4. ✅ Fine-grained authorization is working correctly

The OpenFGA authorization model successfully enforces column-level access control!